In [2]:
#basic imports
import pandas as pd
import numpy as np

#fitbit packages 
import fitbit
from python_fitbit import gather_keys_oauth2 as Oauth2

#datetime 
import datetime

import time
import json

import glob
import os

In [66]:
#set globals
#date I joined Fitbit
date_joined_fitbit = '2016-05-21'
today = datetime.datetime.now().date()
dates = pd.date_range(start = date_joined_fitbit, end = today, freq = 'MS')
days = pd.date_range(start = date_joined_fitbit, end = today, freq = 'D')
client = authorize_fitbit_app()

In [89]:
def authorize_fitbit_app():
    '''
    Authorize the script to access my Fitbit data
    and return fitbit python object
    '''
    #app id
    client_id = '2287X8'
    #app password
    client_secret = '2dfcb152b035ca0bdff3d99abc6af66e'
    #connect to server for access and refresh tokens
    server = Oauth2.OAuth2Server(client_id, client_secret)
    server.browser_authorize()
    ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
    REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
    #instantiate fitbit object
    client = fitbit.Fitbit(client_id, client_secret, 
                           oauth2=True, access_token=ACCESS_TOKEN, 
                           refresh_token=REFRESH_TOKEN)
    return client

def get_weight(dates):
    '''
    Get historical weight data from Fitbit.
    Note: I log my weight using the MyFitnessPal app, which Fitbit
    downloads from. Because MyFitnessPal does not have a public API, I would
    have to download my weight statistics manually. This script allows me to
    programmatically access that data through Fitbit's connection to MyFitnessPal
    I also use this instead of the time series fitbit method because time series
    method imputes days where no weight was entered as the last known date. This way,
    I can manually interpolate the interim days' weight myself.
    '''
    #create empty list
    weight = []
    #iterate through period index of frequency month
    for date in dates:
        #create dataframe of weight data starting from the first of the month
        #to the end of the month
        df = pd.DataFrame(client.get_bodyweight(base_date = date, period = '1m')['weight'])
        #append to list
        weight.append(df)
    #concatenate into dataframe and export
    weight = pd.concat(weight, ignore_index = True, sort = False)
    weight.to_csv('weight.csv', index = False)
    #return for console inspection
    return weight



def get_sleep(days):
    sleep_list = []
    for day in days:
        json = client.get_sleep(day)
        stats = pd.DataFrame(json['sleep'])
        summary = pd.DataFrame(json['summary'], index = [0])
        df = pd.concat([stats, summary], axis = 1)
        sleep_list.append(df)
        time.sleep(2)
    sleep = pd.concat(sleep_list, ignore_index = True, sort = False)
    sleep.to_csv('sleep.csv', index = False)
    return sleep

def get_heart_rate_data():
    '''
    get heart rata data at sub-minute granularity on workout days,
    updating for days I'm missing
    '''
    

    '''identify days for which no heart rate data is downloaded'''
    #import lifting data 
    lifts = pd.read_csv('lifts.csv', index_col = 'date')
    #get unique workout dates as datetime index as of the first date I joined fitbit
    lifting_days = pd.to_datetime(lifts.index).intersection(days).unique()
    
    #get list of heart rate files
    current_files = glob.glob('heart_rate_data/*.json')
    #create index of filelist and replace all non-digit character, leaving only the date
    downloaded_days = pd.Index(current_files).str.replace('\D+', '')
    #convert to datetime index 
    downloaded_days = pd.to_datetime(downloaded_days)
    
    #get dates 
    dates_to_download = lifting_days.difference(downloaded_days)
    
    '''download intraday heart rate data for all undownloaded days'''
    for counter, day in enumerate(dates_to_download, 1):
        #convert date from timestamp to string
        day = str(day.date())
        #get intraday heart rate data at second granularity
        heart_rate = client.intraday_time_series('activities/heart', 
                                                base_date= day, 
                                                detail_level='1sec')
        #save to json
        with open(f'heart_rate_data/heart_rate_{day}.json', 'w') as outfile:
            json.dump(heart_rate, outfile)
        #if request number reacehs a multiple of 100, sleep for an hour to let the
        #api reset. Fitbit's api has a rate limit of 150 requests per hour
        #which resets at the top of each hour, not necessarily an hour
        #after reaching th elimit
        if counter % 100 == 0:
            print('Sleeping for an hour at {}')
            #Sleeping for an hour
            time.sleep(3600)

In [90]:
get_heart_rate_data()